# Abby Baskind
# HW2
# 16 Sep 2023

# Import the usual packages

In [10]:
# Basic packages
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr
import pandas as pd
import math
import netCDF4 as nc

# DateTime packages
from matplotlib.dates import DateFormatter
from datetime import datetime, timedelta
import time
import matplotlib.dates as mdates

# Stats packages
import scipy
import bottleneck as bn
import PyCO2SYS as pyco2
import gsw
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import seaborn as sns

# Logistical packages
import requests
from importlib import reload
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.offsetbox import AnchoredText

I have some functions and API credentials saved in my `phyto` directory, so get those.

In [11]:
%cd '/Users/akbaskind/Documents/Python Scripts/phyto'
%run PLT.py

/Users/akbaskind/Documents/Python Scripts/phyto


# Set up API retrieval and get some data
I'm getting some data I use frequently for my thesis, just so I have something to work with. The data originally comes from Andy Davies Hydrocat sensor near Jamestown. It has gone through rigorous quality control and has data from Jan 1, 2022 to sometime in April 2023.

In [12]:
#The scope is always look like this so we did not need to change anything

import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Name of our Service Account Key
google_key_file = 'servicecredentials.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

# Spreadsheet key
spreadsheet_key = '1uLRAvFDsTSfS-c45DiG-W6VMa8I1J7XD1O-pJA4fp10'

# Worksheet keys
wks_hydrocat620 = 'Hydrocat 620'

#Opening the worksheet by using Worksheet ID
workbook = gc.open_by_key(spreadsheet_key)

## HYDROCAT 620
#Selecting which sheet to pulling the data
sheet = workbook.worksheet(wks_hydrocat620)
values = sheet.get_all_values()
hydrocat620 = pd.DataFrame(values[1:], columns = values[0])


for k in hydrocat620.keys():
    if k == 'DateTime' or k == 'time':
        hydrocat620[k] = pd.to_datetime(hydrocat620[k])
    else:
        hydrocat620[k] = pd.to_numeric(hydrocat620[k])

In [13]:
hydrocat620

,DateTime,Temperature [degC],Salinity [PSU],DO [mg/L],raw pH [total],Fluorescence [ug-chl/L],pH final,pH (normalized to 15°C)
0,2022-01-04 09:31:15.400,6.5078,30.7871,9.592,7.934479,7.478,7.934479,7.797240
1,2022-01-04 10:46:15.400,6.3631,30.8278,9.663,7.934814,1.844,7.934814,7.795139
2,2022-01-04 11:01:15.200,6.3063,30.7459,9.633,7.934773,2.373,7.934773,7.794166
3,2022-01-04 11:16:15.200,6.2367,30.7015,9.680,7.934821,3.158,7.934821,7.793057
4,2022-01-04 11:31:15.200,6.2593,30.6492,9.698,7.924688,2.170,7.924688,7.783583
...,...,...,...,...,...,...,...,...
34421,2023-04-30 22:46:29.000,11.7928,28.4188,8.547,7.971222,0.021,7.971222,7.920264
34422,2023-04-30 23:01:29.000,11.7991,28.4178,8.472,7.961210,0.020,7.961210,7.910442
34423,2023-04-30 23:16:29.000,11.8380,28.3284,8.471,7.971013,0.021,7.971013,7.920790
34424,2023-04-30 23:31:29.000,11.9170,28.1962,8.406,7.970686,0.020,7.970686,7.921743


# Function 1: `pd.DataFrame.resample()` from https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html
* my dataframe is `hydrocat620` and has data every 15 minutes (for the most part, except where some data was excluded)
* The goal here is to get datasets with different temporal resolutions (e.g. daily, weekly, monthly). While this function does have the capacity to resample annually, I don't have enough data to justify that.

In [19]:
daily = 'D'
weekly = 'W'
monthly = 'M'
hourly = '60min'
H620_daily = hydrocat620.resample(daily, on='DateTime').mean()
H620_weekly = hydrocat620.resample(weekly, on='DateTime').mean()
H620_monthly = hydrocat620.resample(monthly, on='DateTime').mean()
H620_hourly = hydrocat620.resample(hourly, on='DateTime').mean()

Checking the output of monthly, everything looks good.

In [28]:
H620_monthly

,Temperature [degC],Salinity [PSU],DO [mg/L],raw pH [total],Fluorescence [ug-chl/L],pH final,pH (normalized to 15°C)
DateTime,,,,,,,
2022-01-31,3.000744,30.133280,11.614558,8.112014,9.477543,8.112014,7.909716
2022-02-28,3.540695,29.026253,11.211418,8.187245,3.566157,8.187245,7.993140
2022-03-31,5.635667,28.967569,10.316234,8.089050,2.219137,8.089050,7.933647
2022-04-30,8.529931,29.257438,9.490863,8.010033,2.400573,8.010033,7.905038
2022-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-30,19.870675,30.388744,7.358862,7.788738,4.307867,7.788738,7.861307
2022-07-31,24.097148,30.803195,6.774081,7.754948,3.798538,7.754948,7.887851
2022-08-31,24.227186,31.197291,5.723571,7.741671,4.939357,7.741671,7.876129
2022-09-30,21.559271,30.792587,6.399515,7.778392,1.626582,7.778392,7.874971


# Function 2: standard error
Standard error is a bit more complicated that just the standard deviation. It involves, first, calculating the standard deviation with a 1 degree of freedom. Next, we need to know $n$ of each sample. Then, we have to take the sqaure root of $n$.
$$ \text{SEM} = \frac{\sigma}{\sqrt{n}}$$

We begin by taking the standard deviation using `numpy.std()`. Documentation is here: https://numpy.org/doc/stable/reference/generated/numpy.std.html.

In [31]:
STD = hydrocat620.resample(monthly, on='DateTime').std(ddof = 1)

We then need to figure out the $n$ of each sample. We can use `pandas.DataFrame.count()` which is here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html.

In [32]:
n = hydrocat620.resample(monthly, on='DateTime').count()

Next, we can take the square root of $n$ using `numpy.sqrt()` which is here: https://numpy.org/doc/stable/reference/generated/numpy.sqrt.html.

In [33]:
sqrt_n = np.sqrt(n)

If we put it all together as

```
SEM = STD/sqrt_n
```

we get our standard error of the mean.

In [34]:
STD/sqrt_n

,DO [mg/L],DateTime,Fluorescence [ug-chl/L],Salinity [PSU],Temperature [degC],pH (normalized to 15°C),pH final,raw pH [total]
DateTime,,,,,,,,
2022-01-31,0.037504,NaN,0.157167,0.021914,0.051485,0.002690,0.003506,0.003506
2022-02-28,0.023704,NaN,0.059184,0.036512,0.023018,0.002224,0.002597,0.002597
2022-03-31,0.009998,NaN,0.045038,0.017629,0.022161,0.000876,0.000887,0.000887
2022-04-30,0.005912,NaN,0.059447,0.017589,0.031781,0.000723,0.000596,0.000596
2022-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-30,0.006798,NaN,0.058723,0.004771,0.020515,0.000669,0.000744,0.000744
2022-07-31,0.012157,NaN,0.045937,0.004553,0.026238,0.001757,0.001550,0.001550
2022-08-31,0.011223,NaN,0.063557,0.001852,0.013802,0.000944,0.000868,0.000868
2022-09-30,0.020075,NaN,0.028845,0.008806,0.028957,0.001452,0.001661,0.001661


Alternatively, we can do this all in one line of code.

In [37]:
hydrocat620.resample(monthly, on='DateTime').std(ddof = 1)/np.sqrt(hydrocat620.resample(monthly, on='DateTime').count())

,DO [mg/L],DateTime,Fluorescence [ug-chl/L],Salinity [PSU],Temperature [degC],pH (normalized to 15°C),pH final,raw pH [total]
DateTime,,,,,,,,
2022-01-31,0.037504,NaN,0.157167,0.021914,0.051485,0.002690,0.003506,0.003506
2022-02-28,0.023704,NaN,0.059184,0.036512,0.023018,0.002224,0.002597,0.002597
2022-03-31,0.009998,NaN,0.045038,0.017629,0.022161,0.000876,0.000887,0.000887
2022-04-30,0.005912,NaN,0.059447,0.017589,0.031781,0.000723,0.000596,0.000596
2022-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-30,0.006798,NaN,0.058723,0.004771,0.020515,0.000669,0.000744,0.000744
2022-07-31,0.012157,NaN,0.045937,0.004553,0.026238,0.001757,0.001550,0.001550
2022-08-31,0.011223,NaN,0.063557,0.001852,0.013802,0.000944,0.000868,0.000868
2022-09-30,0.020075,NaN,0.028845,0.008806,0.028957,0.001452,0.001661,0.001661


We can also write this as a function. This function is definitely not the most user friendly, but it does enough for the assignment.

In [35]:
def stderr(df, t):
    """
    t is the period we are resampling our data set.
    df is the original dataset.
    """
    DF = df.resample(t, on='DateTime')
    std = DF.std(ddof = 1)
    sqrt_n = np.sqrt(DF.count())
    return std/sqrt_n
    

In [36]:
stderr(hydrocat620, 'M')

,DO [mg/L],DateTime,Fluorescence [ug-chl/L],Salinity [PSU],Temperature [degC],pH (normalized to 15°C),pH final,raw pH [total]
DateTime,,,,,,,,
2022-01-31,0.037504,NaN,0.157167,0.021914,0.051485,0.002690,0.003506,0.003506
2022-02-28,0.023704,NaN,0.059184,0.036512,0.023018,0.002224,0.002597,0.002597
2022-03-31,0.009998,NaN,0.045038,0.017629,0.022161,0.000876,0.000887,0.000887
2022-04-30,0.005912,NaN,0.059447,0.017589,0.031781,0.000723,0.000596,0.000596
2022-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-30,0.006798,NaN,0.058723,0.004771,0.020515,0.000669,0.000744,0.000744
2022-07-31,0.012157,NaN,0.045937,0.004553,0.026238,0.001757,0.001550,0.001550
2022-08-31,0.011223,NaN,0.063557,0.001852,0.013802,0.000944,0.000868,0.000868
2022-09-30,0.020075,NaN,0.028845,0.008806,0.028957,0.001452,0.001661,0.001661
